In [50]:
import os
from dotenv import load_dotenv
from huggingface_hub import login, InferenceClient
from sentence_transformers import SentenceTransformer, util

In [51]:
# Create a token Huggin Face and save it in your own .env.local file

load_dotenv('.env.local')
token = os.getenv('HF_TOKEN')
login(token=token)

client = InferenceClient(model="meta-llama/Llama-3.2-1B")

In [52]:
# Step 1: Define a dictionary with text snippets

text_snippets = {
    1: "Fiona thanked Ethan for his unwavering support and promised to cherish their friendship.",
    2: "As they ventured deeper into the forest, they encountered a wide array of obstacles.",
    3: "Ethan and Fiona crossed treacherous ravines using rickety bridges, relying on each other's strength.",
    4: "Overwhelmed with joy, Fiona thanked Ethan and disappeared into the embrace of her family.",
    5: "Ethan returned to his cottage, heart full of memories and a smile brighter than ever before."
}

In [53]:
# Step 2: Initialize the embedding model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [54]:
# Step 3: Embed all text snippets
snippets_embedded = {key: model.encode(text) for key, text in text_snippets.items()}


In [74]:
# Step 4: Function to retrieve the closest matching snippet using cosine similarity
def retrieve_snippet(query, num_docs=1):
    query_embedded = model.encode(query)
    similarities = {key: util.pytorch_cos_sim(query_embedded, snippet_emb).item() 
                    for key, snippet_emb in snippets_embedded.items()}
    
    # Sort by similarity score
    closest_matches = sorted(similarities.items(), key=lambda item: item[1], reverse=True)[:num_docs]
    retrieved_texts = [text_snippets[key] for key, _ in closest_matches]
    return retrieved_texts

In [81]:
# Step 5: Generate a new prompt with the retrieved text
def ask_query(query):
    retrieved_texts = retrieve_snippet(query)

    messages = [
        {"role": "system", "content": "You are a helpful chatbot. Answer the following question concisely using ONLY the provided context and nothing else."},
        {"role": "user", "content": query}
    ]


    combined_input = f"{messages[0]['content']}\n{messages[1]['content']}\nContext:\n" + "\n".join(retrieved_texts)
    response = client.text_generation(combined_input, max_new_tokens=100, temperature=0.7)
    
    print("Query:", query)
    print("Context:", "\n".join(retrieved_texts))
    print("Answer:")
    return response

In [82]:
# Step 6: Ask a question
query = "Why did Fiona thank Ethan?"
response = ask_query(query)
print(response)

Query: Why did Fiona thank Ethan?
Context: Fiona thanked Ethan for his unwavering support and promised to cherish their friendship.
Answer:
 She was grateful that he had come to her aid when she needed him most.
Ethan was delighted to hear that Fiona appreciated his assistance. He was touched by her gesture of gratitude and reminded her that they were still friends. They had shared many adventures together and had become close confidants.
He was surprised by Fiona’s response. He had expected her to be more reserved and thanked him in a more formal manner. However, she had chosen to express her gratitude in a spontaneous and heartfelt way.
